# NPCC Contingency Generation

This file is used to generate the Gen Trip figures.

In [1]:
import numpy as np
import pandas as pd

import andes
import ams

import datetime

import matplotlib
import matplotlib.pyplot as plt

In [2]:
matplotlib.rcdefaults()

In [3]:
%matplotlib inline

In [4]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'andes:{andes.__version__}')
print(f'ams:{ams.__version__}')

Last run time: 2024-04-26 14:32:24
andes:1.9.1.post47+g6e178a22
ams:0.9.6.post5+gef1d8a0


In [5]:
andes.config_logger(stream_level=50)
ams.config_logger(stream_level=50)

In [6]:
ams_file = './data/npcc_uced_base.xlsx' # npcc_uced_base
andes_file = './data/npcc_ibr.xlsx' # npcc_base, npcc_ibr

k_load = 1.05 # load factor, 0.97, 1.05

out_file = './res/npcc_ibr_105_out.csv' # npcc_base_h_out.csv, npcc_ibr_h_out.csv, npcc_ibri_h_out
# base: SynGen with regular cost curve
# ibr: IBR with zero cost curve, 
# ibri: IBR with same cost curve as base case

ad_case = out_file.split('/')[-1].split('.')[0].split('_')[1]

In [7]:
sp = ams.load(ams_file,
              setup=True,
              no_output=True,
              default_config=True,)

idx_pq = sp.PQ.idx.v
p0_pq = sp.PQ.p0.v.copy()
q0_pq = sp.PQ.q0.v.copy()

sp.PQ.alter(src='p0', idx=idx_pq, value=p0_pq * k_load)
sp.PQ.alter(src='q0', idx=idx_pq, value=q0_pq * k_load)

In [8]:
sa = sp.to_andes(addfile=andes_file,
                 setup=True, no_output=True, default_config=True)

In [9]:
def test_andes(gen_idx):
    """
    gen_idx: str
        Tripped generator idx
    """
    sa = sp.to_andes(addfile=andes_file,
                    setup=True, no_output=True, default_config=True)

    # set the tripped gen
    sa.Toggle.set(src='dev', attr='v', idx='Toggler_1', value=gen_idx)

    tg_idx = sa.TGOV1NDB.idx.v
    sa.TGOV1NDB.alter(src='VMAX', idx=tg_idx, value=999*np.ones(sa.TGOV1NDB.n))
    sa.TGOV1NDB.alter(src='VMIN', idx=tg_idx, value=np.zeros(sa.TGOV1NDB.n))

    iex_idx = sa.IEEEX1.idx.v
    vrmax = sa.IEEEX1.VRMAX.v.copy()
    vrmin = sa.IEEEX1.VRMIN.v.copy()
    k_iex = 500
    sa.IEEEX1.alter(src='VRMAX', idx=iex_idx, value=k_iex * vrmax)
    sa.IEEEX1.alter(src='VRMIN', idx=iex_idx, value=k_iex * vrmin)

    ist_idx = sa.IEEEST.idx.v
    lsmax = sa.IEEEST.LSMAX.v.copy()
    lsmin = sa.IEEEST.LSMIN.v.copy()
    k_ist = 500
    sa.IEEEST.alter(src='LSMAX', idx=ist_idx, value=k_ist * lsmax)
    sa.IEEEST.alter(src='LSMIN', idx=ist_idx, value=k_ist * lsmin)

    # for ibri, do not change the cost curve of VSG
    if ad_case == 'ibr':
        vsg_idx = sa.REGCV1.gen.v
        gcost_vsg_idx = sp.GCost.find_idx(keys='gen', values=vsg_idx)
        sp.GCost.set(src='c2', attr='v', idx=gcost_vsg_idx, value=np.zeros(sa.REGCV1.n))
        sp.GCost.set(src='c1', attr='v', idx=gcost_vsg_idx, value=np.zeros(sa.REGCV1.n))
        sp.GCost.set(src='c0', attr='v', idx=gcost_vsg_idx, value=np.zeros(sa.REGCV1.n))
    
    sp.RTED.init()
    sp.RTED.run(solver='MOSEK')
    sp.RTED.dc2ac()

    sp.dyn.send(routine='RTED', adsys=sa)

    sa.PFlow.run()

    _ = sa.TDS.init()

    sa.TDS.config.criteria = True
    sa.TDS.config.no_tqdm = True

    sa.TDS.config.tf = 50
    sa.TDS.run()

    stg_idx = sp.dyn.link['stg_idx'].tolist()
    stg_bus = sa.StaticGen.get(src='bus', attr='v', idx=stg_idx)
    stg_bus_uid = sa.Bus.idx2uid(stg_bus)

    busfreq_bus = sa.BusFreq.find_idx(keys='bus', values=stg_bus)
    busfreq_bus_uid = sa.Bus.idx2uid(stg_bus)

    Tf = sa.BusFreq.Tf.v.mean().round(3)
    Tw = sa.BusFreq.Tw.v.mean().round(3)

    fig, ax = plt.subplots(1, 2, figsize=(10, 5), dpi=80)

    fmin, fmax=59.9, 60.1
    vmin, vmax=0.8, 1.4
    trip_gen = sa.Toggle.dev.v[0]

    sa.TDS.plt.plot(sa.GENCLS.omega,
                    ax=ax[0], fig=fig,
                    grid=True, show=False, legend=False,
                    ytimes=sa.config.freq, ymin=fmin, ymax=fmax,
                    title=f'{ad_case}, k_load={k_load}')
    sa.TDS.plt.plot(sa.GENROU.omega,
                    ax=ax[0], fig=fig,
                    grid=True, show=False, legend=False,
                    ytimes=sa.config.freq,)
    sa.TDS.plt.plot(sa.REGCV1.omega,
                    ax=ax[0], fig=fig,
                    grid=True, show=False, legend=False,
                    ytimes=sa.config.freq,)

    sa.TDS.plt.plot(sa.Bus.v,
                    a=stg_bus_uid,
                    ax=ax[1], fig=fig,
                    grid=True, show=False,
                    ymin=vmin, ymax=vmax,
                    title=f'Trip {trip_gen}')
    sa.TDS.plt.export_csv(out_file)
    plt.close(fig)
    fig.savefig(f'./res/{ad_case}_{int(k_load*100)}_{trip_gen}.png', format='png')
    return sa

In [10]:
syg_idx = sa.GENROU.idx.v + sa.GENCLS.idx.v
syg_idx = ['GENROU_57']
for syg in syg_idx:
    sa = test_andes(syg)

<Toggle Toggler_1>: SynGen.GENROU_57 status changed to 0 at t=1.0 sec.
